# wps_update_metadata

#### wps_update_metadata is a process that runs the [update_metadata](https://github.com/pacificclimate/climate-explorer-data-prep#update_metadata-update-metadata-in-a-netcdf-file) module of PCIC Climate Explorer Data Preparation Tools. Here, the client will try to connect to a remote Thunderbird instance using the url parameter.

In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

from birdy import WPSClient
from pkg_resources import resource_filename
import os
from wps_tools.testing import get_target_url
from netCDF4 import Dataset
from tempfile import NamedTemporaryFile
from wps_tools.output_handling import auto_construct_outputs, nc_to_dataset

# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "thunderbird":
    os.chdir('../')

In [2]:
# NBVAL_IGNORE_OUTPUT
url = get_target_url("thunderbird")
print(f"Using thunderbird on {url}")

Using thunderbird on https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thunderbird/wps


In [3]:
thunderbird = WPSClient(url)

#### Help for individual processes can be diplayed using the ? command (ex. bird.process?).

In [4]:
# NBVAL_IGNORE_OUTPUT
thunderbird.update_metadata?

Signature:
thunderbird.update_metadata(
    netcdf=None,
    loglevel='INFO',
    updates_file=None,
    updates_string=None,
)
Docstring:
Update file containing missing, invalid, or incorrectly named global or variable metadata attributes

Parameters
----------
netcdf : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    NetCDF file
updates_file : ComplexData:mimetype:`text/x-yaml`
    The filepath of an updates file that specifies what to do to the metadata it finds in the NetCDF file
updates_string : string
    The string in yaml format that specifies what to do to the metadata it finds in the NetCDF file
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level

Returns
-------
output : ComplexData:mimetype:`application/x-netcdf`
    Output Netcdf File
File:      ~/code/birds/thunderbird/</tmp/thunderbird-venv/lib/python3.8/site-packages/birdy/client/base.py-2>
Type:      method


#### We can use the docstring to ensure we provide the appropriate parameters.

In [5]:
# run update_metadata with yaml text and opendap netcdf inputs
updates_string = '''
global:
    - institute_id:
    - institute_ID: PCIC
    - address: University House 1 
    - Institution: <- institution
'''
opendap_netcdf = "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/storage/data/projects/comp_support/daccs/test-data/gdd_annual_CanESM2_rcp85_r1i1p1_1951-2100.nc"
opendap_output = thunderbird.update_metadata(updates_string = updates_string, netcdf = opendap_netcdf)
opendap_output_data = opendap_output.get(asobj=True)[0]

In [6]:
# run update_metadata with yaml file and local netcdf inputs
updates_file = resource_filename('tests', 'metadata-conversion/simple-conversion.yaml')
local_netcdf = resource_filename('tests', 'data/gdd_annual_CanESM2_rcp85_r1i1p1_1951-2100.nc')
local_output = thunderbird.update_metadata(updates_file = updates_file, netcdf = local_netcdf)
local_output_data = local_output.get(asobj=True)[0]

#### Once the process has completed we can extract the results and ensure it is what we expected.

In [7]:
input_data = Dataset("https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/storage/data/projects/comp_support/daccs/test-data/gdd_annual_CanESM2_rcp85_r1i1p1_1951-2100.nc")

# Expected output metadata determined from the input yaml file
# global:
#     - institute_id:
#     - institute_ID: PCIC
#     - address: University House 1 
#     - Institution: <- institution
expected = {
    "institute_ID": "PCIC",
    "address": "University House 1",
    "Institution": input_data.institution,
}

In [8]:
def test_metadata(output_data):
    # updated metadata
    metadata = {
        "institute_ID": output_data.institute_ID,
        "address": output_data.address,
        "Institution": output_data.Institution,
    }

    assert metadata == expected

In [9]:
test_metadata(opendap_output_data)
test_metadata(local_output_data)